###### Central Southeast Grasslands

This is a draft indicator for the 2022 base blueprint.



Created by Amy Keister, last run by Amy Keister on June 24, 2022. It took 6 hours and 38 minutes to run.

In [1]:
#import libraries
import os
import arcpy

In [2]:
import time
start = time.time()

In [3]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
SourceWorkspace= r"F:\GIS_DATA\LanduseLandcover\KnownGrasslands"
OutWorkspace = r"E:\WORKING\BaseBlueprint\2022\4_Indicators\InteriorSoutheastGrasslands\InteriorSEGrasslands.gdb"

In [4]:
# define final indicator outputs
SEout = r"E:\WORKING\BaseBlueprint\2022\4_Indicators\InteriorSoutheastGrasslands\InteriorSoutheastGrasslands.tif"

In [5]:
# define sub-indicator outputs
#Known
#PerFrb

In [6]:
# define rasters used for cell size, extent, and snapping
SEraster= r"E:\WORKING\BaseBlueprint\2022\1_SubregionsAndExtent\BaseBlueprintExtent2022.tif"

In [7]:
# define inputs
SubRg= r"E:\WORKING\BaseBlueprint\2022\1_SubregionsAndExtent\BaseBlueprintSubRgn.shp"
TNCgs= r"F:\GIS_DATA\DecisionSupportTools\TNCGeophysicalSettings\Settings_2020_3_31_USAwwat.tif"
chEPV= r"F:\GIS_DATA\LanduseLandcover\CentralHardwoodsEcologicalPotential\raster_data.gdb\ecopot_al_60"
nlcd= r"F:\GIS_DATA\LanduseLandcover\NLCD\NLCD_landcover_2019_release_all_files_20210604\nlcd_2019_land_cover_l48_20210604\nlcd_2019_land_cover_l48_20210604.img"
RcovB421= r"F:\GIS_DATA\LanduseLandcover\RangelandAnalysisPlatform\vegetation-cover-v3-2021.tif/Band_4"
row= r"F:\GIS_DATA\LanduseLandcover\Electric_Power_Transmission_Lines\Electric_Power_Transmission_Lines.shp"
PAD= r"F:\GIS_DATA\ProtectedLands\PAD_US2_1_GDB\PAD_US2_1.gdb\PADUS2_1Combined_Fee_Easement"

## Start Analysis

In [8]:
# Set the workspace where I want the output to go
arcpy.env.workspace = OutWorkspace

In [9]:
print(arcpy.env.workspace)

E:\WORKING\BaseBlueprint\2022\4_Indicators\CentralSEGrasslands\InteriorSEGrasslands.gdb


## Define extent of indicator

In [10]:
# select the subregions where this indicator is being used
arcpy.analysis.Select(SubRg, "SubRg1", "SubRgn_II IN ('Central Gulf Coastal Plain', 'Interior Plateau', 'Mid East Gulf Coastal Plain', 'North Appalachians', 'North Piedmont', 'Ozarks and Plains', 'South Appalachians', 'South Piedmont')")

<Result 'E:\\WORKING\\BaseBlueprint\\2022\\4_Indicators\\CentralSEGrasslands\\InteriorSEGrasslands.gdb\\SubRg1'>

In [11]:
# add field to convert to raster
arcpy.management.CalculateField("SubRg1", "Raster", "1", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")

<Result 'SubRg1'>

In [12]:
# convert to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SEraster, snapRaster=SEraster, cellSize=SEraster):
    arcpy.conversion.FeatureToRaster("SubRg1", "Raster", "SubRg1R", SEraster)

## Known Prairies (Bin 5)

### kml file conversion
I'm converting the kml files to feature classes first using a batch function

In [13]:
# I can't figure out how to get my walk to include kml files
# so I'm making a temporary geodatabase and using a batch import
arcpy.management.CreateFileGDB(SourceWorkspace, "temp", "CURRENT")

<Result 'F:\\GIS_DATA\\LanduseLandcover\\KnownGrasslands\\temp.gdb'>

In [14]:
# use batch import to convert kml files to feature classes
# I couldn't figure out how to write it so that I didn't have to re-write the full output path :(

arcpy.intelligence.BatchImportData(SourceWorkspace, r"F:\GIS_DATA\LanduseLandcover\KnownGrasslands\temp.gdb", "*.kml", "SUBFOLDERS", "GROUNDOVERLAY")

<Result 'F:\\GIS_DATA\\LanduseLandcover\\KnownGrasslands\\temp.gdb'>

### Feature classes

In [15]:
# Set the workspace where the known south atlantic prairies are stored
arcpy.env.workspace = SourceWorkspace

In [16]:
print(arcpy.env.workspace)

F:\GIS_DATA\LanduseLandcover\KnownGrasslands


In [17]:
#look in the source data for known praries, make a list of all the polygon feature classes
FCList = []

walk = arcpy.da.Walk(SourceWorkspace, datatype="FeatureClass", type="Polygon")

for dirpath, dirnames, filenames in walk:
    # disregard all the additional non-prarie polygons that came with the prairie ridge data
    if "prairie_ridge_final_data.gdb" in dirnames:
        dirnames.remove('prairie_ridge_final_data.gdb')
    for filename in filenames:
        FCList.append(os.path.join(dirpath, filename))

In [18]:
#Print the list made in the above step to make sure it is correct
print(FCList)

['F:\\GIS_DATA\\LanduseLandcover\\KnownGrasslands\\KnownGrasslandsSGI\\NashvilleBasinLimestoneGlades.shp', 'F:\\GIS_DATA\\LanduseLandcover\\KnownGrasslands\\KnownPrairiePatchesInTheGCPO\\Prairies_AL_AR_MO_MS.shp', 'F:\\GIS_DATA\\LanduseLandcover\\KnownGrasslands\\KnownPrairieSouthAtlantic\\CVP\\CVP.shp', 'F:\\GIS_DATA\\LanduseLandcover\\KnownGrasslands\\KnownPrairieSouthAtlantic\\Mecklenburg_Piedmont_prairies\\EarlySuccessional.shp', 'F:\\GIS_DATA\\LanduseLandcover\\KnownGrasslands\\KnownPrairieSouthAtlantic\\NCBG_managedprairies\\NCBG_Managed_Fields.shp', 'F:\\GIS_DATA\\LanduseLandcover\\KnownGrasslands\\KnownPrairieSouthAtlantic\\NC_PrairieRidge\\PrairieRidge_UpdatedPrairies.shp', 'F:\\GIS_DATA\\LanduseLandcover\\KnownGrasslands\\KnownPrairieSouthAtlantic\\NC_TurnipseedNaturePreserveAndQualityROWs\\OS_poly.shp', 'F:\\GIS_DATA\\LanduseLandcover\\KnownGrasslands\\KnownPrairieSouthAtlantic\\SBG_Grass_Restoration\\SBG_Grass_Restoration.shp', 'F:\\GIS_DATA\\LanduseLandcover\\KnownGrasslan

In [19]:
# Change the workspace to where I am saving the outputs
arcpy.env.workspace = OutWorkspace

In [20]:
# Print the current workspace to make sure I'm in the right spot
print(arcpy.env.workspace)

E:\WORKING\BaseBlueprint\2022\4_Indicators\CentralSEGrasslands\InteriorSEGrasslands.gdb


In [21]:
# I keep getting an error when I try to union, so I'm going to try to repair geometery
for FC in FCList:
    arcpy.management.RepairGeometry(FC, "DELETE_NULL", "ESRI")

In [22]:
# combine known prairie into one layer
arcpy.analysis.Union(FCList, "KnownPrairieUnion")

<Result 'E:\\WORKING\\BaseBlueprint\\2022\\4_Indicators\\CentralSEGrasslands\\InteriorSEGrasslands.gdb\\KnownPrairieUnion'>

In [23]:
# delete the temprorary geodatabase that stored the converted kml files
arcpy.management.Delete(r"F:\GIS_DATA\LanduseLandcover\KnownGrasslands\temp.gdb", '')

<Result 'true'>

### PADUS 2.1 Combined Fee, Easement

In [24]:
# clip pad to the extent of this indicator
arcpy.analysis.PairwiseClip(PAD, "SubRg1", "PADclip", None)

<Result 'E:\\WORKING\\BaseBlueprint\\2022\\4_Indicators\\CentralSEGrasslands\\InteriorSEGrasslands.gdb\\PADclip'>

In [25]:
# select polygons from the PAD US that are designated as grassland reserve program
arcpy.analysis.Select("PADclip", "PADgrp", "Loc_Ds = 'GRP' Or Unit_Nm LIKE '%Grasslands Reserve Program%' Or Unit_Nm LIKE '%Grassland Reserve Program%'")

<Result 'E:\\WORKING\\BaseBlueprint\\2022\\4_Indicators\\CentralSEGrasslands\\InteriorSEGrasslands.gdb\\PADgrp'>

## Combine known grassland layers


In [26]:
# union the known grasslands made above
arcpy.analysis.Union("PADgrp #;KnownPrairieUnion #", "KnownGrasslands", "ALL", None, "GAPS")

<Result 'E:\\WORKING\\BaseBlueprint\\2022\\4_Indicators\\CentralSEGrasslands\\InteriorSEGrasslands.gdb\\KnownGrasslands'>

In [27]:
# add and calculate a field that we can use to convert to a raster, give a value of 5
arcpy.management.CalculateField("KnownGrasslands", "raster", "5", "PYTHON3", '', "SHORT")

<Result 'KnownGrasslands'>

In [28]:
# convert PAD grassland reserve polygons to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SEraster, snapRaster=SEraster, cellSize=SEraster):
    arcpy.conversion.FeatureToRaster("KnownGrasslands", "raster", "KnownGrasslandsR", SEraster)

## wrangle NLCD to make masks for some of these bins

From the known prairie buffer areas, we are going to remove high and medium urban and water from the NLCD

From the potentially compatible management areas we are going to remove low, medium, and high urban, along with a 1 pixel buffer around those classes. Then we will further limit the potentially compatible management class by only keeping the following NLCD classes: 'Developed, Open Space', 'Barren Land', 'Deciduous Forest', 'Evergreen Forest', 'Mixed Forest', 'Shrub/Scrub', 'Herbaceous', 'Hay/Pasture'

### make mask for known prairie buffer area

In [29]:
# Make a raster with the 2 highest urban classes and water as 0 and everything else is 1
with arcpy.EnvManager(outputCoordinateSystem=sr, extent="SubRg1R", snapRaster=SEraster, cellSize=SEraster):
    out_raster = arcpy.sa.Con(nlcd, 0, 1, "Value IN (23,24,11)"); out_raster.save("NLCDBee")

### make mask for potentially compatible management area

In [30]:
# Make a raster with the 3 urban classes as 0 and everything else is 1
with arcpy.EnvManager(outputCoordinateSystem=sr, extent="SubRg1R", snapRaster=SEraster, cellSize=SEraster):
    out_raster = arcpy.sa.Con(nlcd, 0, 1, "Value IN (22,23,24)"); out_raster.save("Urb")

In [31]:
# expand to capture pixels that are within one pixel of a Developed High, Medium or Low intensity pixel in the NLCD. 
# maked all developed pixels and pixels withi 1 pixel of developed 0, everything else is 1
# this can be used as a mask to remove these areas from the potential compatible managment layers
out_raster = arcpy.sa.Expand("Urb", 1, 0); out_raster.save("UrbE")

In [32]:
# make another "restorable landcver class" mask to further restrict the potentially compatible habitat class
# restorable is 1 and everything else is 0
with arcpy.EnvManager(outputCoordinateSystem=sr, extent="SubRg1R", snapRaster=SEraster, cellSize=SEraster):
    out_raster = arcpy.sa.Con(nlcd, 1, 0, "Value IN (21,31,41,42,43,52,71,81)"); out_raster.save("Restorable")

## Buffer known grasslands (Bin 4)
research on bee habitat suggests an 800 meter buffer should be good

In [33]:
arcpy.analysis.Buffer("KnownGrasslands", "KnownGrasslandsBuff", "800 Meters", "FULL", "ROUND", "NONE", None, "PLANAR")

<Result 'E:\\WORKING\\BaseBlueprint\\2022\\4_Indicators\\CentralSEGrasslands\\InteriorSEGrasslands.gdb\\KnownGrasslandsBuff'>

In [34]:
# add and calculate a field that we can use to convert to a raster, give a value of 4
arcpy.management.CalculateField("KnownGrasslandsBuff", "raster4", "4", "PYTHON3", '', "SHORT")

<Result 'KnownGrasslandsBuff'>

In [35]:
# convert buffered known grasslands to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SEraster, snapRaster=SEraster, cellSize=SEraster):
    arcpy.conversion.FeatureToRaster("KnownGrasslandsBuff", "raster4", "KnownGrasslandsBuffR", SEraster)

In [36]:
# remove pixels that are within one pixel of a developed or water pixel in the NLCD using a times
with arcpy.EnvManager(outputCoordinateSystem=sr, extent="SubRg1R", snapRaster=SEraster, cellSize=SEraster):
    out_raster = arcpy.sa.Times("KnownGrasslandsBuffR", "NLCDBee"); out_raster.save("KnownGrasslandsBuffR1") 

## Grassland geology layer (contributes to Bins 1, 2 and 3)

### Find grassland associated pixels in the TNC geophysical settings

In [37]:
# clip TNC data to this indicator extent
out_raster = arcpy.sa.ExtractByMask(TNCgs, "SubRg1R"); out_raster.save("TNCgs1")

In [38]:
# here I am trying to best match the results Rua got with the eastern dataset
# I'm doing a first cut using the CL_GEOSOIL class
with arcpy.EnvManager(outputCoordinateSystem=sr, extent="SubRg1R", snapRaster=SEraster, cellSize=SEraster):
    out_raster = arcpy.sa.Con("TNCgs1", 1, 0, "CL_GEOSOIL IN ('Calcareous Sedimentary','Mafic/Intermediate Granitic', 'Silt/Clay over Limestone', 'Ultramafic')"); out_raster.save("TNC_GeoSoil")

In [39]:
# here I am trying to best match the results Rua got with the eastern dataset
# I'm adding some missing areas using the Full_Name class
with arcpy.EnvManager(outputCoordinateSystem=sr, extent="SubRg1R", snapRaster=SEraster, cellSize=SEraster):
    out_raster = arcpy.sa.Con("TNCgs1", 1, 0, "Full_Name IN ('Calcareous Sedimentary','Moderately Calcareous Sedimentary','Mid Elevation Acidic Sedimentary')"); out_raster.save("TNC_FullName")

In [40]:
# combine the 2 above using cells statistics max
out_raster = arcpy.sa.CellStatistics("TNC_GeoSoil;TNC_FullName", "MAXIMUM", "DATA", "SINGLE_BAND"); out_raster.save("TNCmatch")

### Find grassland associated pixels in the central hardwoods potential veg

In [41]:
# reclassify the Central Hardwoods Ecological Potential Vegitation to pull out grassland associated classes
# in this geodatabase raster, the values in the VEG_ORDER field related to grass are as follows:
# 1 = Prairie/Grassland, 2 = Prairie/Savanna  (< 20% canopy), 3 = Prairie/Savanna (Barrens)  (< 20% canopy),  
# 4 = Glade/Savanna Mosaic (< 20% canopy), 5 = Oak Open Woodland  (20-50% canopy), 7 = Pine/Bluestem Open Woodland (20-50% canopy)
with arcpy.EnvManager(outputCoordinateSystem=sr, extent="SubRg1R", snapRaster=SEraster, cellSize=SEraster):
    out_raster = arcpy.sa.Reclassify(chEPV, "VEG_ORDER", "1 1;2 1;3 1;4 1;5 1;6 0;7 1;8 0;9 0;10 0;11 0", "DATA"); out_raster.save("chPVgrass")

In [42]:
# clip Central Hardwoods grassland assocated classes to this indicator extent
#out_raster = arcpy.sa.ExtractByMask("chPVgrass", "SubRg1R"); out_raster.save("chPVgrass1")

### Combine TNC and CH grassland associated classes

In [43]:
# Use a mosaic and pick which layer gets used first
arcpy.management.MosaicToNewRaster("chPVgrass;TNCmatch", OutWorkspace, "GrassGeology", None, "8_BIT_UNSIGNED", None, 1, "FIRST", "FIRST")

<Result 'E:\\WORKING\\BaseBlueprint\\2022\\4_Indicators\\CentralSEGrasslands\\InteriorSEGrasslands.gdb\\GrassGeology'>

In [44]:
# the above output was striped so I take times 1
out_raster = arcpy.sa.Times("GrassGeology", 1); out_raster.save("GrassGeology1")

## Potentially compatible management layer (contributes to bins 2 and 3)

### Power line right-of-way
this does yet not include variable buffering of transmission lines based on voltage

In [45]:
# make a copy of the lines for edits
arcpy.management.CopyFeatures(row, "row", '', None, None, None)

<Result 'E:\\WORKING\\BaseBlueprint\\2022\\4_Indicators\\CentralSEGrasslands\\InteriorSEGrasslands.gdb\\row'>

In [46]:
# add and calculate a field that we can use to convert to a raster
arcpy.management.CalculateField("row", "dissolve", "3", "PYTHON3", '', "SHORT")

<Result 'row'>

In [47]:
# convert lines to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, extent="SubRg1R", snapRaster=SEraster, cellSize=SEraster):
    arcpy.conversion.FeatureToRaster("row", "dissolve", "rowr", SEraster)

### perenial forb and grass from range cover layers
we thought about trying to use 5 years of data to see if the area remained in rangeland over 5 years, but didn't have time this year.

In [48]:
# extract band 4 - perennial forb and grass
# This didn't work when I tried to add environments to reproject while extracting
arcpy.management.CopyRaster(RcovB421, "PerFrbGrs21", '', None, "255", "NONE", "NONE", '', "NONE", "NONE", '', "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

<Result 'E:\\WORKING\\BaseBlueprint\\2022\\4_Indicators\\CentralSEGrasslands\\InteriorSEGrasslands.gdb\\PerFrbGrs21'>

In [49]:
# reproject the perennial forb and grass raster
with arcpy.EnvManager(outputCoordinateSystem=sr, extent="SubRg1R", snapRaster=SEraster, cellSize=SEraster):
    arcpy.management.ProjectRaster("PerFrbGrs21", "PerFrbGrs21a", 'PROJCS["NAD_1983_Contiguous_USA_Albers",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-96.0],PARAMETER["Standard_Parallel_1",29.5],PARAMETER["Standard_Parallel_2",45.5],PARAMETER["Latitude_Of_Origin",23.0],UNIT["Meter",1.0]]', "NEAREST", "30.2197497164463 30.2197497164463", "WGS_1984_(ITRF00)_To_NAD_1983", None, 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', "NO_VERTICAL")

In [50]:
# pull out rangeland pixels with perennial forb and gras cover greater than 30%
with arcpy.EnvManager(outputCoordinateSystem=sr, extent="SubRg1R", snapRaster=SEraster, cellSize=SEraster):
    out_raster = arcpy.sa.Con("PerFrbGrs21a", 3, 0, "VALUE > 30"); out_raster.save("PerFrbGrs21aGT30")

### Combine the transmission lines and rangeland potentially compatible habitat


In [51]:
# use cell statistics to combine
out_raster = arcpy.sa.CellStatistics("rowr;PerFrbGrs21aGT30", "MAXIMUM", "DATA", "SINGLE_BAND"); out_raster.save("PotCompMang")

### Limit potentially compatible habitat using NLCD

In [52]:
# remove pixels that are within one pixel of low, medium, or high development
with arcpy.EnvManager(outputCoordinateSystem=sr, extent="SubRg1R", snapRaster=SEraster, cellSize=SEraster):
    out_raster = arcpy.sa.Times("PotCompMang", "UrbE"); out_raster.save("PotCompMang2") 

In [53]:
# Limit to restorable landcover types
with arcpy.EnvManager(outputCoordinateSystem=sr, extent="SubRg1R", snapRaster=SEraster, cellSize=SEraster):
    out_raster = arcpy.sa.Times("PotCompMang2", "Restorable"); out_raster.save("PotCompMang3") 

In [54]:
# reclassify to remove 0s for region group
out_raster = arcpy.sa.Reclassify("PotCompMang3", "Value", "0 NODATA;1 3;3 3", "DATA"); out_raster.save("PotCompMang4")

In [55]:
# use a region group to remove isolated pixels from compatible managment
out_raster = arcpy.sa.RegionGroup("PotCompMang4", "EIGHT", "WITHIN", "ADD_LINK", 0); out_raster.save("PotCompMang4Rg")

In [56]:
# use lookup to access the count field from the previous raster, this lets us see how big each clump is
out_raster = arcpy.sa.Lookup("PotCompMang4Rg", "Count"); out_raster.save("count1")

In [57]:
# pull out clumps less than 3 pixels 
# need to re-run this because I changed from out value of 1 to 3 to make the minus work
out_raster = arcpy.sa.Con("count1", 3, 0, "VALUE < 3"); out_raster.save("small")

In [58]:
# reclassify to change nodata to 0
out_raster = arcpy.sa.Reclassify("small", "Value", "NODATA 0;0 0;3 3", "DATA"); out_raster.save("small0")

In [59]:
# remove the clumps less than 3 pixels from the potentially compatible management layer
out_raster = arcpy.sa.Minus("PotCompMang3", "small0"); out_raster.save("PotCompMang5") 

### Make layers for bins 1, 2, and 3


In [60]:
# take potentially compatible managment times grassland geology 
# this gives me potentially compatible manamgent in grassland as value 3
out_raster = arcpy.sa.Times("PotCompMang5", "GrassGeology1"); out_raster.save("PotCompINGeo") 

In [61]:
# this results in potentially compatible manamgnet outside grassland as value 3, need to reclass
out_raster = arcpy.sa.Con("GrassGeology1", "PotCompMang5", 0, "VALUE = 0"); out_raster.save("PotCompOUTGeo")

In [62]:
# reclass
out_raster = arcpy.sa.Reclassify("PotCompOUTGeo", "Value", "0 0;3 2", "DATA"); out_raster.save("PotCompOUTGeo2")

## Combine all classes


In [63]:
# use cell statistics to combine
out_raster = arcpy.sa.CellStatistics("KnownGrasslandsR;KnownGrasslandsBuffR1;PotCompInGeo;PotCompOUTGeo2;GrassGeology1", "MAXIMUM", "DATA", "SINGLE_BAND"); out_raster.save("indicator")

### Finalize indiator

do final steps for all indicators to add description fields, clip and export to SE extent, clip and export to SA extent

In [64]:
# set code block for next step
codeblock = """
def Reclass(v):
    if v == 5:
        return '5 = Known grassland'
    elif v == 4:
        return '4 = Known grassland buffer'
    elif v == 3:
        return '3 = Potentially compatible management within grassland geology (undeveloped powerline right-of-way or perennial forbs and grasses)'
    elif v == 2:
        return '2 = Potentially compatible management outside of grassland geology (undeveloped powerline right-of-way or perennial forbs and grasses)'
    elif v == 1:
        return '1 = Grassland geology'
    elif v == 0:
        return '0 = Grassland less likely'
"""

In [65]:
# add and calculate description field to hold indicator values
arcpy.management.CalculateField("indicator", "descript", "Reclass(!value!)", "PYTHON3", codeblock, "TEXT")

<Result 'E:\\WORKING\\BaseBlueprint\\2022\\4_Indicators\\CentralSEGrasslands\\InteriorSEGrasslands.gdb\\indicator'>

In [66]:
# set code block for next step
codeblock = """
def Reclass1(Value):
	if Value == 5:
		return 115
	if Value == 4:
		return 168
	if Value == 3:
		return 255
	if Value == 2:
		return 255
	if Value == 1:
		return 255
	if Value == 0:
		return 255
	else:
		return 255
		
def Reclass2(Value):
	if Value == 5:
		return 76
	if Value == 4:
		return 112
	if Value == 3:
		return 170
	if Value == 2:
		return 211
	if Value == 1:
		return 235
	if Value == 0:
		return 255
	else:
		return 255
		
def Reclass3(Value):
	if Value == 5:
		return 0
	if Value == 4:
		return 0
	if Value == 3:
		return 0
	if Value == 2:
		return 127
	if Value == 1:
		return 175
	if Value == 0:
		return 255
	else:
		return 255
"""

In [67]:
# calculate Red field
arcpy.management.CalculateField("indicator", "Red", "Reclass1(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Green field
arcpy.management.CalculateField("indicator", "Green", "Reclass2(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Blue field
arcpy.management.CalculateField("indicator", "Blue", "Reclass3(!Value!)", "PYTHON3", codeblock, "SHORT")

<Result 'E:\\WORKING\\BaseBlueprint\\2022\\4_Indicators\\CentralSEGrasslands\\InteriorSEGrasslands.gdb\\indicator'>

In [68]:
# clip to the exent of this indicator
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=SEraster, cellSize=SEraster):
    out_raster = arcpy.sa.ExtractByMask("indicator", "SubRg1R"); out_raster.save("SEMask2")

In [8]:
# export as .tif 
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=SEraster, cellSize=SEraster):
    arcpy.management.CopyRaster("SEMask2", SEout, '', None, "255", "NONE", "NONE", "16_BIT_UNSIGNED", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

In [70]:
# this prints the time it took this notebook to run in seconds
end = time.time()
print(end - start)

23921.42526960373
